In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
%%time
%pip install catboost
%pip install optuna
#%pip install optuna_distributed
#%pip install openfe
%pip install seaborn
%pip install xgboost
%pip install lightgbm
%pip install fastkaggle
#%pip install h2o
%pip install -Uqq fastbook
#%pip install polars
#%pip install -q -U autogluon.tabular
#%pip install autogluon
#%pip install --upgrade pip
%pip install tqdm
#%pip install wandb
#%pip install sweetviz

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [2998 lines of output]
      Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.6" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_system == "AIX"' don't match you

In [ ]:
#hide
!pip install fastkaggle
!pip install -Uqq fastbook

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm

import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *

In [7]:
#hide
import fastbook
fastbook.setup_book()
from fastbook import *

ModuleNotFoundError: No module named 'fastbook'

# Under The Hood: Training a digit classifier

Below i take chapter 4 of the fastai book where we are shown the fundamentals of deep learning and re do it to show the concepts laid out there in my own understanding. 

First i shall redo the original example, then do it using a different dataset to show an understanding of the same concepts applied to a different example.

In this chapter, we discuss arrays, tensors and how to use the above data structures expressively through broadcasting. We also take a deep dive into stochastic gradient descent, a method our models can use for learning by automatically adjusting weights in the right direction.

Furthermore, i take a look at choosing a loss function and the role of mini batches while showing the mathematics behind a neural network.

Finally we put all the above together but first we take a look at how images are represented to a computer. For this example, we use the mnist dataset, a dataset of handwritten digits from Yann Le Cun and his colleagues. Specifically we shall be trying to classify whether an image is a 3 or a 7.





# Pixels

In order to understand what happens in a computer vision model, we start by seeing how images are represented to a computer which is through pixels.

First we download the dataset of numbers for 3 and 7 which we shall be using in this tutorial

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)

NameError: name 'untar_data' is not defined

In [4]:
#hide
Path.BASE_PATH = path

NameError: name 'path' is not defined

I use the built in fastai method .ls which returns an object of a special class L in fastai which has all of python's list functionality inbuilt plus some extra features such as returning the count of items before printing the items.

In [ ]:
path.ls()

The downloaded dataset includes both folders for training and testing which consist of images of 3 and 7, which we call the labels or targets in this dataset as we see below.

In [ ]:
path'/train'.ls()

In [ ]:
threes = path'/train/3'.ls().sorted()
threes

Lets take a look at one of the images using the Image class from the Python Imaging Library (PIL)/ Pillow which is a fork of the original PIL library.

This is the most common library for opening, manipulating and viewing images.Jupyter has built in support for PIL, so it displays the image automatically when we call the image object as shown below.

In [ ]:
im3_path = threes[1]
im3 = Image.open(im3_path)
im3

It is important to take note that in a computer, everything is represented as a number. We can view the numbers that make up an image by converting the image to numbers as we shall see below.

In [ ]:
array(im3)[4:10,4:10]

Above we pass in a the image and view part of it, specifically the rows from index 4 (included) to 10 (not included) and the same for the columns.NumPy indexes from top to bottom and left to right, so this section is located in the top-left corner of the image. We can do the same thing for a tensor.

In [ ]:
tensor(im3)[4:10,4:10]

To clearly visualize how an image is created from numbers, we slice an array to pick the part with top of the image and color code the values in a pandas dataframe using gradient where darker cells represent higher values and vice versa.

In [ ]:
im3_t = tensor(im3)
df = pd.Dataframe(im3_t[4:15,4:22])
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

# Conclusion

I am going to directly quote the below from the book, this has motivated me in multiple ways on my continous deep learning Journey. I guess sometimes it helps to remind myself that rome wasn't built in a day.

> # Sidebar: Tenacity and Deep Learning
> The story of deep learning is one of tenacity and grit by a handful of dedicated researchers. After early hopes (and hype!) neural networks went out of favor in the 1990's and 2000's, and just a handful of researchers kept trying to make them work well. Three of them, Yann Lecun, Yoshua Bengio, and Geoffrey Hinton, were awarded the highest honor in computer science, the Turing Award (generally considered the "Nobel Prize of computer science"), in 2018 after triumphing despite the deep skepticism and disinterest of the wider machine learning and statistics community.
> 
> Geoff Hinton has told of how even academic papers showing dramatically better results than anything previously published would be rejected by top journals and conferences, just because they used a neural network. Yann Lecun's work on convolutional neural networks, which we will study in the next section, showed that these models could read handwritten text—something that had never been achieved before. However, his breakthrough was ignored by most researchers, even as it was used commercially to read 10% of the checks in the US!
> 
> In addition to these three Turing Award winners, there are many other researchers who have battled to get us to where we are today. For instance, Jurgen Schmidhuber (who many believe should have shared in the Turing Award) pioneered many important ideas, including working with his student Sepp Hochreiter on the long short-term memory (LSTM) architecture (widely used for speech recognition and other text modeling tasks, and used in the IMDb example in <<chapter_intro>>). Perhaps most important of all, Paul Werbos in 1974 invented back-propagation for neural networks, the technique shown in this chapter and used universally for training neural networks (Werbos 1994). His development was almost entirely ignored for decades, but today it is considered the most important foundation of modern AI.
> 
> There is a lesson here for all of us! On your deep learning journey you will face many obstacles, both technical, and (even more difficult) posed by people around you who don't believe you'll be successful. There's one guaranteed way to fail, and that's to stop trying. We've seen that the only consistent trait amongst every fast.ai student that's gone on to be a world-class practitioner is that they are all very tenacious.